In [1]:
import pandas as pd
import sys
import os
from pathlib import Path
import sqlite3

project_root = os.path.abspath('../../')
if project_root not in sys.path:
    sys.path.append(project_root)
    
from src.data.db import DBConnection, execute_query, get_pitcher_data
from config import DBConfig

In [2]:
class NotebookDBConnection(DBConnection):
    def __init__(self):
        db_path = os.path.join(project_root, 'data', 'pitcher_stats.db')
        super().__init__(db_name=db_path)

In [3]:
pd.set_option('display.max_columns', 150)

In [4]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print([table[0] for table in tables])

['pitcher_ids', 'sqlite_sequence', 'teams', 'pitcher_mapping', 'statcast_pitchers', 'team_batting', 'statcast_batters']


In [20]:
with NotebookDBConnection() as conn:
    df_batters = pd.read_sql_query("SELECT * FROM statcast_batters LIMIT 25000;", conn)
    df_pitchers = pd.read_sql_query("SELECT * FROM statcast_pitchers LIMIT 25000;", conn)
    df_team_batting = pd.read_sql_query("SELECT * FROM team_batting;", conn)
    

In [22]:
df_pitchers.shape

(25000, 115)

In [24]:
with NotebookDBConnection() as conn:
    cursor = conn.cursor()
    cursor.execute("PRAGMA table_info(statcast_pitcher)")
    print(cursor.fetchall())
    

[(0, 'pitch_type', 'TEXT', 0, None, 0), (1, 'game_date', 'TEXT', 0, None, 0), (2, 'release_speed', 'REAL', 0, None, 0), (3, 'release_pos_x', 'REAL', 0, None, 0), (4, 'release_pos_z', 'REAL', 0, None, 0), (5, 'player_name', 'TEXT', 0, None, 0), (6, 'batter', 'INTEGER', 0, None, 0), (7, 'pitcher', 'INTEGER', 0, None, 0), (8, 'events', 'TEXT', 0, None, 0), (9, 'description', 'TEXT', 0, None, 0), (10, 'spin_dir', 'INTEGER', 0, None, 0), (11, 'spin_rate_deprecated', 'INTEGER', 0, None, 0), (12, 'break_angle_deprecated', 'INTEGER', 0, None, 0), (13, 'break_length_deprecated', 'INTEGER', 0, None, 0), (14, 'zone', 'INTEGER', 0, None, 0), (15, 'des', 'TEXT', 0, None, 0), (16, 'game_type', 'TEXT', 0, None, 0), (17, 'stand', 'TEXT', 0, None, 0), (18, 'p_throws', 'TEXT', 0, None, 0), (19, 'home_team', 'TEXT', 0, None, 0), (20, 'away_team', 'TEXT', 0, None, 0), (21, 'type', 'TEXT', 0, None, 0), (22, 'hit_location', 'INTEGER', 0, None, 0), (23, 'bb_type', 'TEXT', 0, None, 0), (24, 'balls', 'INTEGER'